# Homework #3: PCA/Hyperparameter/CV
Data source: http://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data

In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np

In [2]:
data = arff.loadarff('4year.arff')
df = pd.DataFrame(data[0])

In [3]:
df['bankruptcy'] = (df['class']==b'1')
df.drop(columns=['class'], inplace=True)
df.columns = ['X{0:02d}'.format(k) for k in range(1,65)] + ['bankruptcy']

In [4]:
df.describe()

,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,...,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64
count,9791.000000,9791.000000,9791.000000,9749.000000,9.771000e+03,9791.000000,9791.000000,9773.000000,9792.000000,9791.000000,...,9.792000e+03,9771.000000,9791.000000,9776.000000,9791.000000,9178.000000,9760.000000,9.771000e+03,9749.000000,9561.000000
mean,0.043019,0.596404,0.130959,8.136600,6.465164e+01,-0.059273,0.059446,19.884016,1.882296,0.389040,...,7.686330e+03,-0.992263,0.035022,1.133287,0.856053,118.156064,25.194430,2.015157e+03,8.660813,35.949619
std,0.359321,4.587122,4.559074,290.647281,1.475939e+04,6.812754,0.533344,698.697015,17.674650,4.590299,...,7.605261e+04,77.007971,8.945365,8.038201,26.393305,3230.316692,1099.260821,1.171461e+05,60.838202,483.318623
min,-12.458000,0.000000,-445.910000,-0.045319,-3.794600e+05,-486.820000,-12.458000,-1.848200,-0.032371,-445.910000,...,-7.132200e+05,-7522.100000,-597.420000,-30.892000,-284.380000,0.000000,-12.656000,-1.496500e+04,-0.024390,-0.000015
25%,0.001321,0.263145,0.020377,1.047000,-5.121700e+01,-0.000578,0.003004,0.428300,1.006675,0.294440,...,2.184000e+01,0.003121,0.008768,0.885722,0.000000,5.356325,4.267700,4.323400e+01,2.938800,2.012900
50%,0.041364,0.467740,0.199290,1.591800,-5.557600e-02,0.000000,0.048820,1.088700,1.161300,0.510450,...,9.503300e+02,0.043679,0.098026,0.958305,0.002129,9.482000,6.283550,7.472900e+01,4.848900,4.041600
75%,0.111130,0.689255,0.410670,2.880400,5.573200e+01,0.065322,0.126940,2.691000,1.970225,0.714290,...,4.694550e+03,0.117170,0.242680,0.996163,0.211790,19.506000,9.938200,1.233450e+02,8.363800,9.413500
max,20.482000,446.910000,22.769000,27146.000000,1.034100e+06,322.200000,38.618000,53209.000000,1704.800000,12.602000,...,6.123700e+06,112.020000,226.760000,668.750000,1661.000000,251570.000000,108000.000000,1.077900e+07,5662.400000,21153.000000


In [5]:
sum(df.bankruptcy == True)

515

In [6]:
df.fillna(df.mean(), inplace=True)
df.isna().sum()
X_imp = df.values

In [7]:
from sklearn.model_selection import train_test_split

X, y = X_imp[:, :-1], X_imp[:, -1]
y = y.astype(bool)
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [8]:
import sklearn.preprocessing as skpre

stdsc = skpre.StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)
X_test_std = stdsc.transform(X_test)
print(X_test_std.shape)

(6854, 64)
(2938, 64)


In [9]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

## (1) LR pipeline
hyper-parameter:
1. C: np.logspace(-6,6,100)
2. penalty: "none", "l1", "l2"

### (a) LR with penalties

In [19]:
pipe_lr = make_pipeline(
    PCA(n_components=3),
    GridSearchCV( # grid search for hyper-parameters
        LogisticRegression(
            solver='liblinear',
            class_weight='balanced',
            random_state=1,
        ), 
        param_grid={
            'C': np.logspace(-6,6,100),
            "penalty": ["l1", "l2"]
        }, 
        cv=5, # 5 fold Cross-Validation
        scoring='f1',
        verbose=1
    )
)

pipe_lr.fit(X_train_std, y_train)
param_best = pipe_lr.steps[1][1].best_params_
y_train_pred = pipe_lr.predict(X_train_std)
y_test_pred = pipe_lr.predict(X_test_std)
f1_train = f1_score(y_train, y_train_pred)
f1_test = f1_score(y_test, y_test_pred)
acc_train = accuracy_score(y_train, y_train_pred)
acc_test = accuracy_score(y_test, y_test_pred)

print(
    param_best,
    f1_train,
    acc_train,
    f1_test,
    acc_test
)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
{'C': 0.07054802310718646, 'penalty': 'l2'} 0.1791703442188879 0.7286256200758681 0.17995910020449898 0.7270251872021783


## (2) SVM pipeline
hyper-parameter:
1. C: np.logspace(-2,2,4)
2. gamma: np.logspace(-2,2,4) * 0.5

In [21]:
pipe_svm = make_pipeline(
    PCA(n_components=3),
    GridSearchCV(
        SVC(
            kernel='rbf', 
            tol=1e-2,
            class_weight='balanced',
            random_state=1,
        ), 
        param_grid={
            'C': np.logspace(-2,2,4),
            'gamma': np.logspace(-2,2,4) * 0.5,
        }, 
        cv=5, 
        scoring='f1',
        verbose=1
    )
)

pipe_svm.fit(X_train_std, y_train)
param_best = pipe_svm.steps[1][1].best_params_
y_train_pred = pipe_svm.predict(X_train_std)
y_test_pred = pipe_svm.predict(X_test_std)
f1_train = f1_score(y_train, y_train_pred)
f1_test = f1_score(y_test, y_test_pred)
acc_train = accuracy_score(y_train, y_train_pred)
acc_test = accuracy_score(y_test, y_test_pred)
print(
    f1_train,
    acc_train,
    f1_test,
    acc_test
)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
0.18621307072515664 0.7347534286548001 0.1873684210526316 0.7372362151123213


## (3) Decision Tree pipeline
hyper-parameter:
1. criterion: ["gini", "entropy"]
2. max_depth: range(1, 10)

In [22]:
pipe_tree = make_pipeline(
    PCA(n_components=3),
    GridSearchCV(
        DecisionTreeClassifier(
            class_weight='balanced',
            random_state=1,
        ), 
        param_grid={
            "criterion": ["gini", "entropy"],
            "max_depth": range(1, 10)
        }, 
        cv=5, 
        scoring='f1',
        verbose=1
    )
)

pipe_tree.fit(X_train_std, y_train)
param_best = pipe_tree.steps[1][1].best_params_
y_train_pred = pipe_tree.predict(X_train_std)
y_test_pred = pipe_tree.predict(X_test_std)
f1_train = f1_score(y_train, y_train_pred)
f1_test = f1_score(y_test, y_test_pred)
acc_train = accuracy_score(y_train, y_train_pred)
acc_test = accuracy_score(y_test, y_test_pred)
print(
    param_best,
    f1_train,
    acc_train,
    f1_test,
    acc_test
)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
{'criterion': 'gini', 'max_depth': 3} 0.20150501672240803 0.7213306098628538 0.18725099601593623 0.7222600408441117
